# Clustering  and Dimensionality Reduction

# Create example data for KMeans clustering from scratch
First, let's create some example data with 4 clusters using make_blobs.

We set random_state=1 so we all get the same clusters.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Generating the sample data from make_blobs
# This particular setting has one distict cluster and 3 clusters placed close
# together.
X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1)  # For reproducibility
d1 = X[:,0] # first dimension
d2 = X[:,1] # second dimension
plt.scatter(d1,d2)
plt.show()

# We now write a function that initializes k centroids by randomly selecting them from the data points

In [ ]:
def initialize_centroids(points, k):
    """returns k centroids from the initial points"""
    centroids = points.copy()
    np.random.shuffle(centroids)
    return centroids[:k]

k = 4;

centroids = initialize_centroids(X, k)
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)
plt.show()

# We now write a function that initializes k centroids following kmeans++ approach

In [ ]:
import scipy 
def initialize_centroids_plus_plus(X, K):
    C = [X[0]]
    indexes = [0]
    for k in range(1, K):
        D2 = scipy.array([min([scipy.inner(c-x,c-x) for c in C]) for x in X])
        probs = D2/D2.sum()
        cumprobs = probs.cumsum()
        r = scipy.rand()
        for j,p in enumerate(cumprobs):
            if r < p:
                i = j
                break
        C.append(X[i])
        indexes.append(i)
    return indexes

k = 4;
indexes = initialize_centroids_plus_plus(X, k)
centroids = X[indexes]
print(centroids)
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)
plt.show()

# Now let's define a function that returns the closest centroid for each point. 

In [ ]:
def closest_centroid(points, centroids):
    """returns an array containing the index to the nearest centroid for each point"""
    distances = np.sqrt(((points - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)


# We can test the code like so:

In [ ]:
c = initialize_centroids(X, k)
closest_centroid(X, c)

# The last step in the algorithm is to move the centroids to the mean location associated with it:

In [ ]:
def move_centroids(points, closest, centroids):
    """returns the new centroids assigned from the points closest to them"""
    return np.array([points[closest==k].mean(axis=0) for k in range(centroids.shape[0])])

move_centroids(X, closest_centroid(X, c), c)

# We can visualize the first two steps in the following way:

In [ ]:
plt.subplot()
plt.scatter(X[:, 0], X[:, 1])
indexes = initialize_centroids_plus_plus(X, k)
centroids = X[indexes]
plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)
plt.show()

closest = closest_centroid(X, centroids)
plt.scatter(X[:, 0], X[:, 1], c=closest)
plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)
plt.show()

plt.subplot()
plt.scatter(X[:, 0], X[:, 1])
closest = closest_centroid(X, centroids)
centroids = move_centroids(X, closest, centroids)
plt.scatter(X[:, 0], X[:, 1], c=closest)
plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)
plt.show()




# Now let's define a function that returns the distnace between two vectors

In [ ]:
# Euclidean Distance Caculator
def dist(a, b):
    return np.sqrt(((a - b)**2).sum())
    #return np.linalg.norm(a - b)

# The complete algorithm

In [ ]:
k = 4;
indexes = initialize_centroids_plus_plus(X, k)
centroids = X[indexes]
# To store the value of centroids when it updates
centroids_old = np.zeros(centroids.shape)

error = dist(centroids, centroids_old)
print("Error: ",error)


# Loop will run till the error becomes zero
while error != 0:
    # Storing the old centroid values
    centroids_old = centroids
    # Assigning each value to its closest cluster
    closest = closest_centroid(X, centroids)
    # Finding the new centroids by taking the average value
    centroids = move_centroids(X, closest, centroids)
    # update the error
    error = dist(centroids, centroids_old)  
    print("Error: ", error)

plt.scatter(X[:, 0], X[:, 1], c=closest)
plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)
plt.show()

# KMeans using sklearn


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
import numpy as np
import pandas as pd
from collections import Counter

estimator = KMeans(init='k-means++', n_clusters=4, n_init=10)
estimator.fit(X)
plt.scatter(X[:, 0], X[:, 1])
plt.scatter(estimator.cluster_centers_[:, 0], estimator.cluster_centers_[:, 1], c='r', s=100)
plt.show()

## EXERCISE: Clustering with k-means

[Adapted from http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html#example-cluster-plot-kmeans-digits-py]

### Loading handwritten digits data

We'll work with the handwritten digits dataset, a classic machine-learning dataset used to explore automatic recognition of handwritten digits (i.e., 0, 1, 2, ..., 9).

For more information:
* http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits
* http://scikit-learn.org/stable/tutorial/basic/tutorial.html#loading-an-example-dataset

In [ ]:
%matplotlib inline
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

digits = load_digits()

print('The digits data comprises {} {}-dimensional representations of handwritten digits:\n{}\n'.format(
        digits.data.shape[0],
        digits.data.shape[1],
        digits.data
    ))

print('It also includes labels:\n{}\n'.format(digits.target))

print('And it includes the original 8x8 image representation:\n{}\n'.format(digits.images[0]))

print('Let\'s look at a few images:\n')
NUM_SUBPLOT_ROWS = 1
NUM_SUBPLOT_COLS = 8
for i in range(NUM_SUBPLOT_ROWS*NUM_SUBPLOT_COLS):
    _ = plt.subplot(NUM_SUBPLOT_ROWS,NUM_SUBPLOT_COLS,i+1)
    _ = plt.imshow(digits.images[i], cmap=plt.cm.gray_r, interpolation='nearest')

### Clustering handwritten digits

That's the data. Now let's try clustering these 64d vectors.

`scikit-learn` implements many differnt machine learning algorithms.

The normal pattern is to:
1. intialise an estimator (e.g., `estimator = KMeans()`)
1. fit to the training data (e.g., `estimator.fit(training_data)`)
1. label the test data (e.g., `estimator.predict(test_data)`)

For clustering, we don't have separate training and test data.

So the labelling is created when we fit and accessed by `estimator.labels_`.

`estimator.inertia_` gives the sum of squared errors (SSE).

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
import numpy as np
import pandas as pd
from collections import Counter

# First let's scale the digits data (center to mean and scale to unit variance)
data = scale(digits.data)
print('Scaled digits data:\n{}\n'.format(data))

# Let's grab the data we'll need
n_samples, n_features = data.shape
n_digits = len(np.unique(digits.target)) # classes
labels = digits.target
freq = (Counter(labels))
print ('Label counts for each digit:',sorted(freq.items()))

# And let's run k-means, specifying initialisation (k-means++), k (n_digits),
# and the number of runs (10)
estimator = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
estimator.fit(data)
print('Sum of squared errors:', estimator.inertia_)
print('Clusters from k-means:', estimator.labels_[:10])
print('Gold standard classes:', labels[:10])

y_actu = pd.Series(labels, name='Actual')
y_pred = pd.Series(estimator.labels_, name='Predicted')
confusion = pd.crosstab(y_pred, y_actu)
print(confusion)


In [ ]:
print('Clusters from k-means:', estimator.cluster_centers_)

### TODO Try different initialisations

Initialisation has a large effect on cluster output. Let's try a few options.

* Initialise with random (`init='random'`)
* Run PCA with k components (`pca = PCA(n_components=n_digits).fit(data)`)
* Use PCA components to initialise (`init=pca.components_`)
* Can we determine which approach is best?

In [ ]:
# 1 - 
est_random = KMeans(init='random', n_clusters=n_digits, n_init=10)
est_random.fit(data)
print('RANDOM INITIALISATION')
print('Num of squared errors:', est_random.inertia_)
print('Clusters from k-means:', est_random.labels_[:10])
print('Gold standard classes:', labels[:10])
print('')

# 2 - 
from sklearn.decomposition import PCA
pca = PCA(n_components=n_digits).fit(data)

# 3 - 
est_pca = KMeans(init=pca.components_, n_clusters=n_digits, n_init=1)
est_pca.fit(data)
print('INITIALISATION WITH PCA COMPONENTS')
print('Num of squared errors:', est_pca.inertia_)
print('Clusters from k-means:', est_pca.labels_[:10])
print('Gold standard classes:', labels[:10])
print('')

# 4 - It looks like k-means++ >> random >> PCA from SSE/inertia.
#     But SSE is an internal validation measure.
#     Since we're trying to cluster by digit, we can't really say
#     which is best without comparing to the gold partition.

## EXERCISE: Evaluating clustering

Since we have a gold-standard labels, we can compare our system clustering to the true partition.

`scikit-learn` includes various metrics for this:
* Homogeneity
* Completeness
* V-measure
* Adjusted Rand index (ARI)
* Adjusted mutual information (AMI)
* Silhouette coefficient

For more information:
* http://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation

Let's compare the above clusterings using V-measure.

In [ ]:
from sklearn import metrics
print('k-means++ initialisation:', metrics.v_measure_score(labels, estimator.labels_))


### Comparing initialisations

Let's be a bit more exhastive, comparing initialisations using variuos evaluation metrics.

In [ ]:
from time import time
from sklearn.decomposition import PCA

sample_size = 300

def bench_k_means(estimator, name, data):
    "Calculate various metrics for comparing system clustering to a gold partition"
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

# print table header
print(75 * '_')
print('init         time  inertia    homo   compl  v-meas     ARI     AMI silhouet')
print(75 * '_')

# benchmark k-means++ initialisation
bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)

# benchmark random initialisation
bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random", data=data)

# benchmark PCA initalisation
pca = PCA(n_components=n_digits).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
              name="PCA-based",
              data=data)

print(75 * '_')

### TODO Reading evaluation output

- Which approach performs best? How would you order the other two?
- Do you neighbours get the same result?
- Can we apply statistical significance testing?
- How else can we test reliability?

In [ ]:
# 1 - PCA > random, PCA > k-means++, hard to say for random and k-means++

# 2 - Run multiple times, k-means clustering depends on initialisation and changes

# 3 - Not directly to the clustering output. Clustering is often more of an exploratory tool.

# 4 - A few possibilities..
#     We could evaluate the impact of different clusterings on another task.
#     We could do a bootstrap stability analysis
#     (http://www.r-bloggers.com/bootstrap-evaluation-of-clusters/).
#     We could use cophenitic correlation for hierarchical clustering
#     (https://en.wikipedia.org/wiki/Cophenetic_correlation).

## EXERCISE: Choosing k

### Create example data for choosing k

First, let's create some example data with 4 clusters using make_blobs.

We set `random_state=1` so we all get the same clusters.

In [ ]:
from sklearn.datasets import make_blobs

# Generating the sample data from make_blobs
# This particular setting has one distict cluster and 3 clusters placed close
# together.
X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1)  # For reproducibility
d1 = X[:,0] # first dimension
d2 = X[:,1] # second dimension
_ = plt.scatter(d1,d2)

### Choosing k using silhouette analysis

[Adapted from http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html]

For good clusterings:
* the average silhouette should be close to 1 indicating that points are far away from neighbouring clusters 
* all cluster silhouettes should be close to the average silhouette score

In [ ]:
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score

import matplotlib.cm as cm
range_n_clusters = range(2,6)

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.Spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhoutte score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.Spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors)

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1],
                marker='o', c="white", alpha=1, s=200)

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

    plt.show()


### TODO Choosing k

[Derived from Data Science from Scratch, Chapter 19]

- Plot inertia against k for k from 2 to 6
- What k would you choose for each? Is it the same?
- Does this work on the handwritten digits code? Why / why not?

In [ ]:
# 1 -
range_n_clusters = [2, 3, 4, 5, 6]
inertia_values = [KMeans(n_clusters=k).fit(X).inertia_ for k in range_n_clusters]
_ = plt.plot(range_n_clusters, inertia_values)

# 2 - From this plot, it looks like the knee is at 4 or maybe 3


In [ ]:

# 3 - Nope. The handwritten digits data is difficult to cluster.
#     However, we haven't done anything clever with our feature representation.
#     We might do better, e.g., with spectral clustering
#     (http://scikit-learn.org/stable/modules/clustering.html#spectral-clustering).
#     Q: Does spectral clustering outperform 
#     We leave this as an extra exercise for the keen reader.
range_n_clusters = range(5,15)
inertia_values = [KMeans(n_clusters=k).fit(data).inertia_ for k in range_n_clusters]
_ = plt.plot(range_n_clusters, inertia_values)

# EXERCISE: Dimensionality Reduction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.preprocessing import scale

## Principal component analysis (PCA)

In [ ]:
iris = datasets.load_iris()


print('The iris data comprises {} {}-dimensional representations of Iris flower.'.format(
        iris.data.shape[0],
        iris.data.shape[1]
    ))



df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris.head()



In [ ]:
df_iris = pd.DataFrame(data= np.c_[iris ['data'], iris ['target']],
                     columns= iris ['feature_names'] + ['target'])

df_iris['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df_iris.head()

In [ ]:
# Let's grab the data we'll need
n_classes = len(np.unique(iris.target_names)) # classes
print("Number of classes = ",n_classes)
labels = iris.target

In [ ]:

#['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
#Plot the initial training points using sepal length vs sepal width

# Plot the training points
def plot_scatter(X,y,labels,colors):
    for i in range(len(colors)):
        px = X[:, 0][y == i]
        py = X[:, 1][y == i]
        plt.scatter(px, py, c=colors[i])
    plt.legend(labels)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.show()

print('Let\'s see how the data looks like:\n')
colors = ['black', 'blue', 'purple']  

plot_scatter(iris.data, iris.target,iris.target_names, colors)

In [ ]:
# let's apply PCA on the data and reduce its dimensionality
pca = PCA()
data = scale(iris.data)
iris_pca = pca.fit_transform(data)
#Explained variance ratio
print(pca.explained_variance_ratio_)


var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3))
print(var)
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.plot(var)
plt.show()
#Cumulative variance
pca.explained_variance_ratio_.sum()

### Deciding how many componenets
The output of the explained varinace shows  that the first component explains 72.8%  of the data set's variation. That means it holds 72.8% of the data's information in one principal component. And by taking the first two components, we only exlude 4.2% of the data set's information and the first two components  contain 95.8% of the iris data set's original information.


In [ ]:

pca = PCA(n_components=2)
data = scale(iris.data)
iris_pca = pca.fit_transform(data)
comps = pd.DataFrame(data = iris_pca, columns = ['pc 1', 'pc 2'])
comps[1:5]

final_comps = pd.concat([comps, pd.DataFrame(data=labels,columns=['target'])], axis = 1)
final_comps[1:5]
#print top five records
final_comps.iloc[1:5,:]

In [ ]:
# Plot the training points
def plot_pca_scatter(X,y,labels,colors):
    for i in range(len(colors)):
        px = X[:, 0][y == i]
        py = X[:, 1][y == i]
        plt.scatter(px, py, c=colors[i])
    plt.legend(labels)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.show()


In [ ]:
# Plot the training points after PCA
colors = ['black', 'blue', 'purple']  
plot_pca_scatter(iris_pca, iris.target,iris.target_names, colors)


### TODO PCA on digits dataset
* Load digits dataset 
* Scale the data
* Apply PCA
* What would be a reasonable number of components for digits  data?

In [ ]:
# TODO: replace the content of this cell with your Python solution
#raise NotImplementedError
digits = datasets.load_digits()

print('The digits data comprises {} {}-dimensional representations of handwritten digits:\n{}\n'.format(
        digits.data.shape[0],
        digits.data.shape[1],
        digits.data
    ))

print('It also includes labels:\n{}\n'.format(digits.target))

print('And it includes the original 8x8 image representation:\n{}\n'.format(digits.images[0]))

In [ ]:
print('Let\'s look at a few images:\n')
NUM_SUBPLOT_ROWS = 1
NUM_SUBPLOT_COLS = 8
for i in range(NUM_SUBPLOT_ROWS*NUM_SUBPLOT_COLS):
    plt.subplot(NUM_SUBPLOT_ROWS,NUM_SUBPLOT_COLS,i+1)
    plt.imshow(digits.images[i], cmap=plt.cm.gray_r, interpolation='nearest')
plt.show()

In [ ]:
# First let's scale the digits data (center to mean and scale to unit variance)
data = scale(digits.data)
print('Scaled digits data:\n{}\n'.format(data))

# Let's grab the data we'll need
n_samples, n_features = data.shape
print("Number of dimensions = ",n_features)
n_digits = len(np.unique(digits.target)) # classes
labels = digits.target
print(digits.target_names)

In [ ]:
pca = PCA()
digits_pca = pca.fit_transform(data)
#Explained variance ratio
pca.explained_variance_ratio_

var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3))
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.plot(var)
plt.show()
#Cumulative variance
pca.explained_variance_ratio_.sum()

In [ ]:
pca = PCA(n_components=40)
digits_pca = pca.fit_transform(data)

In [ ]:
# Plot the training points
def plot_pca_scatter(X_pca,y_digits,labels,colors):
    for i in range(len(colors)):
        px = X_pca[:, 0][y_digits == i]
        py = X_pca[:, 1][y_digits == i]
        plt.scatter(px, py, c=colors[i])
    plt.legend(labels)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.show()

colors = ['black', 'blue', 'purple', 'yellow', 'white', 'red', 'lime', 'cyan', 'orange', 'gray']
   
plot_pca_scatter(digits_pca, labels,digits.target_names, colors)


